In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from keras import layers, losses
from sklearn.model_selection import train_test_split
from keras.models import Model
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import warnings
warnings.filterwarnings('ignore')

from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers
from keras import layers
import keras
import numpy as np

import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
#from sklearn.externals import joblib
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt

from numpy.random import seed
import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR

from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers

from model import get_lstm_model, get_conv_model
from load_dataset import load_dataset
from client_utils import make_logs

2023-11-15 10:20:03.333121: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-15 10:20:03.333137: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [1]:
loss_type = 'mse'
dataset = 'SKAB'
model_name = 'CNN'
test_name = '1_epoch_descentralized_30c'
model_shared = 'All'

models = {}

n_rounds = 20

for round in range(1,n_rounds+1):
    for cid in range(30):

        config = {'server_round':round}

        try:
            x_train,  x_test = load_dataset(dataset_name=dataset, cid = cid, n_clients =34,
                                        server_round = config['server_round'], dataset_size = 60, global_data= False, 
                                        n_components=None)
        except IndexError:
             pass

        if config['server_round'] == 1:
            models[str(cid)] = get_conv_model(x_train)


        model = models[str(cid)]
        model.compile(optimizer='adam', loss= loss_type)

        loss = pd.Series(np.sum(np.mean(np.abs( x_test -  model.predict( x_test)), axis=1), axis=1)).values[0]

        filename = f"logs/{ dataset}/{ model_name}/{ test_name}/evaluate_before_train/loss_{ loss_type}_{ model_shared}.csv"
        make_logs(filename, config, cid =  cid, loss = loss)


        model.compile(optimizer='adam', loss= loss_type)

        n_epochs = 1
        hist =  model.fit( x_train,  x_train,
                epochs = n_epochs, batch_size = 32)
            
        loss = hist.history['loss'][-1]		
        filename = f"logs/{ dataset}/{ model_name}/{ test_name}/train/loss_{ loss_type}_{ model_shared}.csv"
        make_logs(filename, config, cid =  cid, loss = loss)


        models[str(cid)] = model

NameError: name 'load_dataset' is not defined

In [ ]:
models